In [1]:
#-------------------------------------------------------------------------------
# Import libraries
#-------------------------------------------------------------------------------
import ee

#-------------------------------------------------------------------------------
# Trigger the authentication flow
#-------------------------------------------------------------------------------
ee.Authenticate()

#-------------------------------------------------------------------------------
# Initialize the library
#-------------------------------------------------------------------------------
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=tLgxJsTblOc9cBUZpDVdMp1cc0RwbS30lRp_KdBAm_Q&tc=_E_eXRe4kCzUHLC35uskS4iEZDc-n5nlrogiJURBMyI&cc=GhUa0dZOfgaT-sgLa4lhYXjnUEwA2t580XkoitOoTxM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWgavddr8fPlnJUHw92G08wLggnU_VT49MPryGjPr-vXWhCW6zM4hBH40ek

Successfully saved authorization token.


In [2]:
#-------------------------------------------------------------------------------
# Load study area polygon
#-------------------------------------------------------------------------------
geometry = ee.FeatureCollection("projects/ee-my-pafekety-gedi-sentinel1/assets/gedi/study_area")



In [3]:
#-------------------------------------------------------------------------------
# Load ARD tiles
#-------------------------------------------------------------------------------

desired_tiles = ee.FeatureCollection("users/stevenf/cmswest/cms_tiles_alberswgs84").filterBounds(geometry)
tile_list = desired_tiles.aggregate_array('hv').getInfo()
print(tile_list)
len(tile_list)



['002001', '002002', '002003', '002004', '002005', '002006', '003000', '003001', '003002', '003003', '003004', '003005', '003006', '004000', '004001', '004002', '004003', '004004', '004005', '004006', '005001', '005002', '005003', '005004', '005005', '005006', '006001', '006002', '006003', '006004', '006005', '006006', '006007', '007001', '007002', '007003', '007004', '007005', '007006', '007007', '008001', '008002', '008003', '008004', '008005', '008006', '008007', '008008', '009002', '009003', '009004', '009005', '009006', '009007', '009008', '009009', '009010', '009011', '010002', '010003', '010004', '010005', '010006', '010007', '010008', '010009', '010010', '010011', '011002', '011003', '011004', '011005', '011006', '011007', '011008', '011009', '011010', '011011', '012002', '012003', '012004', '012005', '012006', '012007', '012008', '012009', '012010', '012011', '013002', '013003', '013004', '013008', '013009', '013010', '013011', '001004', '001005']


97

In [6]:
#-------------------------------------------------------------------------------
# Set Paramters
#-------------------------------------------------------------------------------

# Time period
desired_year = '2022'
start_date = desired_year + "-06-01"
end_date = desired_year + "-09-30"

# output location
outputFolder = 'gedi_v2_metrics_sar_'+str(desired_year)


print(start_date)
print(end_date)
print(outputFolder)


2022-06-01
2022-09-30
gedi_v2_metrics_sar_2022


In [7]:
#tile_list = ['002004','002005','003005',]

#tile_list = ['009010']
#tile_list = ['002001']
#tile_list = ["011006", "012009", "012010"]

print(str(len(tile_list)) + " Tiles to be processed")

for hv in tile_list:

  #-----------------------------------------------------------------------------
  # Get the desired tile in the list as a earth engine feature 
  #-----------------------------------------------------------------------------
  tile = ee.Feature(desired_tiles.filterMetadata('hv', 'equals', hv).first())
  tile_dimensions = tile.toDictionary().getInfo()
  #print(tile_dimensions)
  #tile_dimensions = {'h': 9, 'hv': '009010', 'maxx': -1083358, 'maxy': 1749390, 'minx': -1105000, 'miny': 1731221, 'v': 10}


  #-----------------------------------------------------------------------------
  # Define output parameters 
  #-----------------------------------------------------------------------------
  crs = 'epsg:5070'
  scale = 30.0
  dimx = int((tile_dimensions['maxx'] - tile_dimensions['minx'])/scale)
  dimy = int((tile_dimensions['maxy'] - tile_dimensions['miny'])/scale)
  dims = str(dimx)+'x'+str(dimy)
  transform = [scale, 0.0, float(tile_dimensions['minx']), 0.0, -scale, float(tile_dimensions['maxy'])]

  #-----------------------------------------------------------------------------
  # Sentinel 1 Metrics 
  #-----------------------------------------------------------------------------

  # Create an image collection of all Sentinel-1 VV images from your input time range, Filter to get images collected in interferometric wide swath mode.
  # VV
  s1_vv = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .select('VV')# \
        #.map(lambda i:  i.updateMask(i.mask().And(i.lt(-30.0).Not())))
          
#s1_vv = ee.ImageCollection('COPERNICUS/S1_GRD')
#        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
#        .filter(ee.Filter.eq('instrumentMode', 'IW'))
#        .select('VV')
#        .map(function(image) {
#          var edge = image.lt(-30.0);
#          var maskedImage = image.mask().and(edge.not());
#          return image.updateMask(maskedImage);
#        });

  def filter_bad(image):
    i = image.select("VH")
    edge = i.lt(-30.0)
    masked_image = i.mask().And(edge.Not())
    return(masked_image)

  # VH
  s1_vh = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .select('VH') \
    .filterMetadata('instrumentMode', 'equals', 'IW') \
    .map(lambda i:  i.updateMask(i.mask().And(i.lt(-30.0).Not()))) 

    


  #-----------------------------------------------------------------------------
  # Calculate Median of time period
  #-----------------------------------------------------------------------------
  # VV
  #s1_vv_median = s1_vv.median().rename("s1_vv_median").clip(tile)
  s1_vv_median = s1_vv.median().rename("s1_vv_median")

  # VH
  #s1_vh_median = s1_vh.median().rename("s1_vh_median").clip(tile)
  s1_vh_median = s1_vh.median().rename("s1_vh_median")
    
  #-----------------------------------------------------------------------------
  # Calculate ratio of VH/VV
  #-----------------------------------------------------------------------------
  s1_vh_vv_ratio = s1_vh_median.divide(s1_vv_median).rename("vh_vv_ratio")

  #-----------------------------------------------------------------------------
  # Calculate ratio of normalized difference
  #-----------------------------------------------------------------------------
  s1_normalized_difference = s1_vh_median.subtract(s1_vv_median).divide(s1_vh_median.add(s1_vv_median)).rename("norm_diff")

  #-----------------------------------------------------------------------------
  # Calculate  Radar Vegetation Index (RVI) = 4 * VH /(VH + VV)
  #-----------------------------------------------------------------------------
  s1_rvi = s1_vh_median.multiply(4).divide(s1_vh_median.add(s1_vv_median)).rename("rvi")

  #-----------------------------------------------------------------------------
  # Compile all predictor variables into a raster stack per tile 
  #-----------------------------------------------------------------------------
  all_predictors = ee.Image.cat(s1_vv_median, s1_vh_median, s1_vh_vv_ratio, s1_normalized_difference, s1_rvi)
  
  #  this worked
  all_predictors_5070 = all_predictors.reproject(crs=crs, crsTransform=[10.0, 0.0, float(tile_dimensions['minx']-30), 0.0, -10.0, float(tile_dimensions['maxy']+30)]) 
  all_predictors_5070_reduced = all_predictors_5070.reduceResolution(reducer= ee.Reducer.mean(), maxPixels=65000)

  #-----------------------------------------------------------------------------
  # Send for loop requests to GEE servers as a set of tasks 
  #-----------------------------------------------------------------------------

  # Assign output tile name
  outname = 'predictor_raster_'+hv+'_'+str(desired_year)
  print(outname)


  # Export file to google drive
  task = ee.batch.Export.image.toDrive(
    image=all_predictors_5070_reduced.float(),
    description=outname,
    fileNamePrefix=outname,
    folder = outputFolder,
    dimensions=dims,
    crs=crs,
    crsTransform=str(transform),
    maxPixels=3e11,
  )
 
  task.start()

97 Tiles to be processed
predictor_raster_002001_2022
predictor_raster_002002_2022
predictor_raster_002003_2022
predictor_raster_002004_2022
predictor_raster_002005_2022
predictor_raster_002006_2022
predictor_raster_003000_2022
predictor_raster_003001_2022
predictor_raster_003002_2022
predictor_raster_003003_2022
predictor_raster_003004_2022
predictor_raster_003005_2022
predictor_raster_003006_2022
predictor_raster_004000_2022
predictor_raster_004001_2022
predictor_raster_004002_2022
predictor_raster_004003_2022
predictor_raster_004004_2022
predictor_raster_004005_2022
predictor_raster_004006_2022
predictor_raster_005001_2022
predictor_raster_005002_2022
predictor_raster_005003_2022
predictor_raster_005004_2022
predictor_raster_005005_2022
predictor_raster_005006_2022
predictor_raster_006001_2022
predictor_raster_006002_2022
predictor_raster_006003_2022
predictor_raster_006004_2022
predictor_raster_006005_2022
predictor_raster_006006_2022
predictor_raster_006007_2022
predictor_raster_0